In [2]:
from typing import TypeVar, Generic, List, Set, Dict, Tuple, Optional, Callable
from functools import reduce

# Type variables
Q = TypeVar('Q')  # State type
S = TypeVar('S')  # Symbol type

# Transition type
Transition = Tuple[Q, Optional[S], Q]

# NFA type
class NFA(Generic[Q, S]):
    def __init__(self, alphabet: List[S], states: List[Q], initial_state: Q, final_states: List[Q], transitions: List[Transition]):
        self.alphabet = alphabet
        self.states = states
        self.initial_state = initial_state
        self.final_states = final_states
        self.transitions = transitions

    def __repr__(self):
        return (f"NFA(alphabet={self.alphabet!r}, states={self.states!r}, \n"
                f"initial_state={self.initial_state!r},\n" f"final_states={self.final_states!r}, \n"
                f"transitions={self.transitions!r})")

# DFA type (similar to NFA, but states are sets of NFA states, and symbols are not optional in transitions)
class DFA(Generic[Q, S]):
    def __init__(self, alphabet: List[S], states: List[List[Q]], initial_state: List[Q], final_states: List[List[Q]], transitions: List[Tuple[List[Q], S, List[Q]]]):
        self.alphabet = alphabet
        self.states = states
        self.initial_state = initial_state
        self.final_states = final_states
        self.transitions = transitions
        
    def __repr__(self):
        return (f"DFA(alphabet={self.alphabet!r}, states={self.states!r},\n "
                f"initial_state={self.initial_state!r}, \n"
                f"final_states={self.final_states!r}, \n"
                f"transitions={self.transitions!r})")
        

# --- Helper functions (akin to Sets Module Functions) ---

def elem(x: Q, a: List[Q]) -> bool:
    """Checks if an element x is present in a list a."""
    return x in a

def diff(a: List[Q], b: List[Q]) -> List[Q]:
    """Returns the set difference between lists a and b (a - b)."""
    return [x for x in a if x not in b]

def insert(x: Q, a: List[Q]) -> List[Q]:
    """Inserts an element x into a list a, maintaining uniqueness."""
    if x in a:
        return a
    else:
        return a + [x]

def insert_all(xs: List[Q], a: List[Q]) -> List[Q]:
    """Inserts all elements from list xs into list a, maintaining uniqueness."""
    for x in xs:
        a = insert(x, a)
    return a

# --- NFA operations ---

def move(nfa: NFA[Q, S], states: List[Q], symbol_opt: Optional[S]) -> List[Q]:
    """Computes the set of states reachable from the given states on the given symbol (or epsilon)."""
    acc = []
    for from_state, sym_opt, to_state in nfa.transitions:
        if elem(from_state, states) and sym_opt == symbol_opt:
            acc = insert(to_state, acc)
    return acc

def e_closure(nfa: NFA[Q, S], states: List[Q]) -> List[Q]:
    """Computes the epsilon closure of the given set of states."""
    def visit(stack: List[Q], visited: List[Q], acc: List[Q]) -> List[Q]:
        if not stack:
            return acc
        node, *rest = stack
        if elem(node, visited):
            return visit(rest, visited, acc)
        else:
            new_visited = insert(node, visited)
            matching_transitions = [
                (from_state, sym_opt, to_state)
                for from_state, sym_opt, to_state in nfa.transitions
                if sym_opt is None and from_state == node
            ]
            neighbors = [to_state for _, _, to_state in matching_transitions]
            new_stack = insert_all(neighbors, rest)
            return visit(new_stack, new_visited, insert(node, acc))

    return sorted(visit(states, [], []))

def explode(s: str) -> List[str]:
    """Converts a string into a list of characters."""
    return list(s)

def accept(nfa: NFA[Q, str], s: str) -> bool:
    """Checks if the NFA accepts the given string."""
    char_lst = explode(s)

    def aux(char_lst: List[str], states: List[Q]) -> List[Q]:
        if not char_lst:
            return states
        sym, *t = char_lst
        next_states = move(nfa, states, sym)
        closure = e_closure(nfa, next_states)
        return aux(t, closure)

    final_closure = aux(char_lst, [nfa.initial_state])
    return any(elem(x, nfa.final_states) for x in final_closure)


# --- NFA to DFA conversion ---
def calculate_transitions(nfa: NFA[Q, S], states: List[Q]) -> List[Tuple[S, List[Q]]]:
    """Calculates transitions for a set of states including the e-closure."""
    closure = e_closure(nfa, states)
    transitions = [
        (symbol, e_closure(nfa, move(nfa, closure, symbol)))
        for symbol in nfa.alphabet
    ]
    return transitions


def new_states(nfa: NFA[Q, S], states: List[Q]) -> List[List[Q]]:
    """Computes the new states that can be reached from the given state set."""
    transitions = calculate_transitions(nfa, states)
    next_states_list = [next_states for _, next_states in transitions if next_states != []]
    unique_next_states = list({frozenset(s) for s in next_states_list})  # Using frozenset for uniqueness
    return [list(s) for s in unique_next_states]  # Convert frozensets back to lists of lists



def new_trans(nfa: NFA[Q, S], states: List[Q]) -> List[Tuple[List[Q], S, List[Q]]]:
    """Computes the new transitions that originate from the given state set."""
    transitions = calculate_transitions(nfa, states)
    closure = e_closure(nfa, states)  # Calculate closure once here
    new_transitions = [
        (closure, symbol, next_states)
        for symbol, next_states in transitions
        if next_states != []
    ]
    return new_transitions

def new_finals(nfa: NFA[Q, S], qs: List[Q]) -> List[List[Q]]:
    """Determines if the given set of states contains any final states."""
    return [qs] if any(elem(state, nfa.final_states) for state in qs) else []


def nfa_to_dfa_step(nfa: NFA[Q, S], dfa: DFA[Q, S], wrk: List[Q], new_states_list: List[List[Q]]) -> DFA[Q, S]:
    """Performs a single step of the NFA to DFA conversion."""
    
    next_trans_list = new_trans(nfa, wrk)
    next_finals_list = new_finals(nfa, wrk)

    updated_states = insert_all(new_states_list, dfa.states)
    updated_transitions = insert_all(next_trans_list, dfa.transitions)
    updated_finals = insert_all(next_finals_list, dfa.final_states)

    return DFA(
        alphabet=dfa.alphabet,
        states=updated_states,
        initial_state=dfa.initial_state,
        final_states=updated_finals,
        transitions=updated_transitions,
    )

def nfa_to_dfa_recursive(nfa: NFA[Q, S], dfa: DFA[Q, S], worklist: List[List[Q]]) -> DFA[Q, S]:
    """Recursively converts an NFA to a DFA."""
    if not worklist:
        return dfa  # Worklist is empty; construction is complete

    wrk, *tail = worklist
    new_states_list = new_states(nfa, wrk) # Calculate new states once here
    new_dfa = nfa_to_dfa_step(nfa, dfa, wrk, new_states_list)
    unvisited_states = diff(new_states_list, dfa.states)
    updated_worklist = insert_all(unvisited_states, tail)
    return nfa_to_dfa_recursive(nfa, new_dfa, updated_worklist)

def nfa_to_dfa(nfa: NFA[Q, S]) -> DFA[Q, S]:
    """Converts an NFA to an equivalent DFA."""
    initial_closure = e_closure(nfa, [nfa.initial_state])
    initial_dfa = DFA(
        alphabet=nfa.alphabet,
        states=[initial_closure],
        initial_state=initial_closure,
        final_states=new_finals(nfa, initial_closure) or [],
        transitions=[],
    )
    return nfa_to_dfa_recursive(nfa, initial_dfa, [initial_closure])

In [3]:
complex_nfa_data = {
    'alphabet': ['a', 'b'],
    'states': [0, 1, 2, 3, 4],
    'initial_state': 0,
    'final_states': [4],
    'transitions': [
        (0, None, 1),
        (0, None, 2),
        (1, 'a', 1),
        (1, 'b', 3),
        (2, 'a', 3),
        (3, None, 4),
        (4, 'a', 4),
        (4, 'b', 4),
    ]
}

# Create an NFA instance
complex_nfa = NFA(
    alphabet=complex_nfa_data['alphabet'],
    states=complex_nfa_data['states'],
    initial_state=complex_nfa_data['initial_state'],
    final_states=complex_nfa_data['final_states'],
    transitions=[
        (from_state, None if sym_opt is None else sym_opt, to_state) 
        for from_state, sym_opt, to_state in complex_nfa_data['transitions']
    ]
)

# Convert the NFA to a DFA
dfa = nfa_to_dfa(complex_nfa)

# Print the resulting DFA
print("DFA:")
print(f"  Alphabet: {dfa.alphabet}")
print(f"  States: {dfa.states}")
print(f"  Initial State: {dfa.initial_state}")
print(f"  Final States: {dfa.final_states}")
print(f"  Transitions:")
for from_state, symbol, to_state in dfa.transitions:
    print(f"    ({from_state}, '{symbol}', {to_state})")


DFA:
  Alphabet: ['a', 'b']
  States: [[0, 1, 2], [3, 4], [1, 3, 4], [4], [1, 4]]
  Initial State: [0, 1, 2]
  Final States: [[3, 4], [1, 3, 4], [4], [1, 4]]
  Transitions:
    ([0, 1, 2], 'a', [1, 3, 4])
    ([0, 1, 2], 'b', [3, 4])
    ([3, 4], 'a', [4])
    ([3, 4], 'b', [4])
    ([1, 3, 4], 'a', [1, 4])
    ([1, 3, 4], 'b', [3, 4])
    ([4], 'a', [4])
    ([4], 'b', [4])
    ([1, 4], 'a', [1, 4])
    ([1, 4], 'b', [3, 4])


In [4]:
complex_nfa_data = {
    'alphabet': ['a'],
    'states': [0, 1, 2],
    'initial_state': 0,
    'final_states': [2],
    'transitions': [
        (0, 'a', 1),
        (1, None, 2)
    ]
}

# Create an NFA instance
complex_nfa = NFA(
    alphabet=complex_nfa_data['alphabet'],
    states=complex_nfa_data['states'],
    initial_state=complex_nfa_data['initial_state'],
    final_states=complex_nfa_data['final_states'],
    transitions=[
        (from_state, None if sym_opt is None else sym_opt, to_state) 
        for from_state, sym_opt, to_state in complex_nfa_data['transitions']
    ]
)

# Convert the NFA to a DFA
dfa = nfa_to_dfa(complex_nfa)

# Print the resulting DFA
print("DFA:")
print(f"  Alphabet: {dfa.alphabet}")
print(f"  States: {dfa.states}")
print(f"  Initial State: {dfa.initial_state}")
print(f"  Final States: {dfa.final_states}")
print(f"  Transitions:")
for from_state, symbol, to_state in dfa.transitions:
    print(f"    ({from_state}, '{symbol}', {to_state})")

DFA:
  Alphabet: ['a']
  States: [[0], [1, 2]]
  Initial State: [0]
  Final States: [[1, 2]]
  Transitions:
    ([0], 'a', [1, 2])


In [7]:
from typing import List, Tuple, Set, Dict, Union, Callable, TypeVar

# Type aliases for better readability
T = TypeVar('T')
State = int
Symbol = str
Transition = Tuple[State, Union[Symbol, None], State]
Regexp = Union[
    "Empty_String", # type: ignore
    Tuple["Char", Symbol], # type: ignore
    Tuple["Union", "Regexp", "Regexp"],
    Tuple["Concat", "Regexp", "Regexp"], # type: ignore
    Tuple["Star", "Regexp"], # type: ignore
]


# Helper functions using sets for efficiency

def add_state(nfa: NFA, state: State) -> NFA:
    return NFA(
        nfa.alphabet,
        list(set(nfa.states) | {state}),
        nfa.initial_state,
        nfa.final_states,
        nfa.transitions,
    )


def add_symbol(nfa: NFA, symbol: Symbol) -> NFA:
    if symbol in nfa.alphabet:
        return nfa
    else:
        return NFA(
            list(set(nfa.alphabet) | {symbol}),
            nfa.states,
            nfa.initial_state,
            nfa.final_states,
            nfa.transitions,
        )


def add_transition(nfa: NFA, transition: Transition) -> NFA:
    return NFA(
        nfa.alphabet,
        nfa.states,
        nfa.initial_state,
        nfa.final_states,
        list(set(nfa.transitions) | {transition}),
    )


def add_f_state(nfa: NFA, state: State) -> NFA:
    return NFA(
        nfa.alphabet,
        nfa.states,
        nfa.initial_state,
        list(set(nfa.final_states) | {state}),
        nfa.transitions,
    )


def add_state_and_trans(nfa: NFA, new_state: State, trans: Transition) -> NFA:
    nfa_with_state = add_state(nfa, new_state)
    return add_transition(nfa_with_state, trans)


# Thompson's Construction Function (Recursive)
def thompson(
    re: Regexp, nfa: NFA, fresh: Callable[[], int]
) -> Tuple[NFA, State, State]:
    if re == "Empty_String":
        # Create two fresh states and add an epsilon transition between them
        i = fresh()
        f = fresh()
        nfa1 = add_state(add_state(nfa, i), f)
        nfa2 = add_transition(nfa1, (i, None, f))
        return (nfa2, i, f)

    elif isinstance(re, tuple) and re[0] == "Char":
        # Create two fresh states, add the character to the alphabet,
        # and add a transition on that character
        _, c = re
        i = fresh()
        f = fresh()
        nfa1 = add_state(add_state(nfa, i), f)
        nfa2 = add_symbol(nfa1, c)
        nfa3 = add_transition(nfa2, (i, c, f))
        return (nfa3, i, f)

    elif isinstance(re, tuple) and re[0] == "Concat":
        # Concatenate two sub-NFAs
        _, r1, r2 = re
        (nfa1, i1, f1) = thompson(r1, nfa, fresh)
        (nfa2, i2, f2) = thompson(r2, nfa1, fresh)
        nfa3 = add_transition(nfa2, (f1, None, i2))
        return (nfa3, i1, f2)

    elif isinstance(re, tuple) and re[0] == "Union":
        # Create a new start and final state, and add epsilon transitions to both sub-NFAs
        _, r1, r2 = re
        (nfa1, i1, f1) = thompson(r1, nfa, fresh)
        (nfa2, i2, f2) = thompson(r2, nfa1, fresh)
        s = fresh()
        t = fresh()
        nfa3 = add_state(add_state(nfa2, s), t)
        nfa4 = add_transition(nfa3, (s, None, i1))
        nfa5 = add_transition(nfa4, (s, None, i2))
        nfa6 = add_transition(nfa5, (f1, None, t))
        nfa7 = add_transition(nfa6, (f2, None, t))
        return (nfa7, s, t)

    elif isinstance(re, tuple) and re[0] == "Star":
        # Apply the Kleene star operation
        _, r = re
        (nfa1, i, f) = thompson(r, nfa, fresh)
        s = fresh()
        t = fresh()
        nfa2 = add_state(add_state(nfa1, s), t)
        nfa3 = add_transition(nfa2, (s, None, i))
        nfa4 = add_transition(nfa3, (s, None, t))
        nfa5 = add_transition(nfa4, (f, None, i))
        nfa6 = add_transition(nfa5, (f, None, t))
        return (nfa6, s, t)
    else:
      raise ValueError(f"Invalid regular expression: {re}")


def regexp_to_nfa(re: Regexp) -> NFA:
    # Initialize a counter for generating unique state IDs
    counter = 0

    # Local fresh function to generate unique state IDs
    def fresh() -> int:
        nonlocal counter
        c = counter
        counter += 1
        return c

    # Start with an empty NFA
    nfa0 = NFA(
        alphabet=[], states=[], initial_state=-1, final_states=[], transitions=[]
    )  # Temporary placeholder for initial_state

    # Build the sub-NFA for the regular expression
    (nfa1, i, f) = thompson(re, nfa0, fresh)

    # Set the actual initial and final states
    return NFA(
        alphabet=nfa1.alphabet,
        states=nfa1.states,
        initial_state=i,
        final_states=[f],
        transitions=nfa1.transitions,
    )

def print_nfa(nfa: NFA) -> None:
    """Prints the NFA's components line by line."""
    print("Alphabet:")
    print(nfa.alphabet)

    print("States:")
    print(nfa.states)

    print(f"Initial State: {nfa.initial_state}")

    print("Final States:")
    print(nfa.final_states)


    print("Transitions:")
    for transition in nfa.transitions:
        print(f"  {transition}")


# Example usage for a(a|b)*:
regexp = ("Concat", ("Char", "a"), ("Star", ("Union", ("Char", "a"), ("Char", "b"))))
nfa = regexp_to_nfa(regexp)
print_nfa(nfa)


Alphabet:
['a', 'b']
States:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Initial State: 0
Final States:
[9]
Transitions:
  (4, 'b', 5)
  (5, None, 7)
  (6, None, 2)
  (7, None, 9)
  (7, None, 6)
  (2, 'a', 3)
  (6, None, 4)
  (1, None, 8)
  (0, 'a', 1)
  (8, None, 6)
  (3, None, 7)
  (8, None, 9)


In [6]:
# Convert the NFA to a DFA
dfa = nfa_to_dfa(nfa)

# Print the resulting DFA
print("DFA:")
print(f"  Alphabet: {dfa.alphabet}")
print(f"  States: {dfa.states}")
print(f"  Initial State: {dfa.initial_state}")
print(f"  Final States: {dfa.final_states}")
print(f"  Transitions:")
for from_state, symbol, to_state in dfa.transitions:
    print(f"    ({from_state}, '{symbol}', {to_state})")

DFA:
  Alphabet: ['a', 'b']
  States: [[0], [1, 2, 4, 6, 8, 9], [2, 4, 5, 6, 7, 9], [2, 3, 4, 6, 7, 9]]
  Initial State: [0]
  Final States: [[1, 2, 4, 6, 8, 9], [2, 4, 5, 6, 7, 9], [2, 3, 4, 6, 7, 9]]
  Transitions:
    ([0], 'a', [1, 2, 4, 6, 8, 9])
    ([1, 2, 4, 6, 8, 9], 'a', [2, 3, 4, 6, 7, 9])
    ([1, 2, 4, 6, 8, 9], 'b', [2, 4, 5, 6, 7, 9])
    ([2, 4, 5, 6, 7, 9], 'a', [2, 3, 4, 6, 7, 9])
    ([2, 4, 5, 6, 7, 9], 'b', [2, 4, 5, 6, 7, 9])
    ([2, 3, 4, 6, 7, 9], 'a', [2, 3, 4, 6, 7, 9])
    ([2, 3, 4, 6, 7, 9], 'b', [2, 4, 5, 6, 7, 9])
